In [70]:
# Importing the API and instantiating the client using your keys
from terra.base_client import Terra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

API_KEY = "sHoKlayVARI2Z6UQQbAFpRWUaumi0S64"
DEV_ID = "hackmitappmit-testing-6KdV9fgTBb"
SECRET = "<Your Signing secret>"



terra = Terra(API_KEY, DEV_ID, SECRET)

In [71]:
parsed_api_response = terra.list_providers().get_parsed_response()
print(parsed_api_response)

ProvidersResponse(status='success', providers=['BIOSTRAP', 'CARDIOMOOD', 'CONCEPT2', 'COROS', 'CRONOMETER', 'DEXCOM', 'DEXCOM_EU', 'DEXCOM_JP', 'EATTHISMUCH', 'EIGHT', 'FATSECRET', 'FINALSURGE', 'FITBIT', 'FREESTYLELIBRE', 'GARMIN', 'GOOGLE', 'HAMMERHEAD', 'HUAWEI', 'IFIT', 'INBODY', 'KOMOOT', 'LIVEROWING', 'LEZYNE', 'MOXY', 'MYFITNESSPAL', 'NUTRACHECK', 'OMRON', 'OMRONUS', 'OURA', 'PELOTON', 'POLAR', 'PUL', 'RIDEWITHGPS', 'ROUVY', 'SUUNTO', 'TECHNOGYM', 'TEMPO', 'TRIDOT', 'TREDICT', 'TRAININGPEAKS', 'TRAINASONE', 'TRAINERROAD', 'UNDERARMOUR', 'VIRTUAGYM', 'WAHOO', 'WEAROS', 'WHOOP', 'WITHINGS', 'XIAOMI', 'XOSS', 'ZWIFT', 'XERT', 'BRYTONSPORT', 'TODAYSPLAN', 'WGER', 'VELOHERO', 'TESTINTEGRATION', 'CYCLINGANALYTICS', 'NOLIO', 'TRAINXHALE', 'KETOMOJOUS', 'KETOMOJOEU', 'STRAVA', 'CLUE', 'HEALTHGAUGE', 'MACROSFIRST', 'SOMNOFY', 'AKTIIA', 'DECATHLON'], sdk_providers=['GOOGLEFIT', 'FREESTYLELIBRESDK', 'APPLE', 'SAMSUNG', 'REALTIME'])


In [14]:
widget_response = terra.generate_widget_session(
    reference_id="USER ID IN YOUR APP",
    providers=[],
    # providers=["FITBIT", "OURA", "FREESTYLELIBRE", "GARMIN", "XIAOMI"],
    auth_success_redirect_url="https://success.url",
    auth_failure_redirect_url="https://failure.url",
    language="en"
).get_parsed_response()

print(widget_response)

WidgetSession(expires_in=900, status='success', session_id='31f0cb3a-2adf-4533-bbdf-8f6a0664504f', url='https://widget.tryterra.co/session/31f0cb3a-2adf-4533-bbdf-8f6a0664504f')


In [ ]:
# User Ids
# GOOGLE: bf1d53bd-0bf4-4f6e-b892-49610a5deda0
# GARMIN: c62f231d-69cf-44f7-ba46-93c9e4de3ccd

In [72]:
# Create a user object
# USER_ID = "bf1d53bd-0bf4-4f6e-b892-49610a5deda0"
USER_ID = "c62f231d-69cf-44f7-ba46-93c9e4de3ccd"
terra_user = terra.from_user_id(USER_ID)

In [83]:
# This block sends the webhook request for data
daily_resp = terra_user.get_daily(start_date=datetime.strptime('2023-08-01','%Y-%m-%d'), end_date=datetime.now(), to_webhook = False, with_samples=False )
daily_resp_json = daily_resp.get_json()

In [84]:
daily_resp_json

{'message': 'Large request submitted. The data is being processed and will be sent to your webhook in chunks',
 'reference': '84163c17-88a5-41a6-b4b4-fe8fd272c7b3',
 'status': 'success',
 'type': 'sent_to_webhook',
 'user': {'last_webhook_update': None,
  'provider': 'GARMIN',
  'reference_id': 'USER ID IN YOUR APP',
  'scopes': 'COURSE_IMPORT,ACTIVITY_EXPORT,HEALTH_EXPORT,MCT_EXPORT,WORKOUT_IMPORT',
  'user_id': 'c62f231d-69cf-44f7-ba46-93c9e4de3ccd'}}

In [53]:
def get_df_data(daily_resp_json):
    avg_heart_rate = []
    act_seconds = []
    act_calories = []
    avg_stress = []
    for day in daily_resp_json["data"]:
        ahr = day["heart_rate_data"]["summary"]["avg_hr_bpm"]
        act_secs = day["active_durations_data"]["activity_seconds"]
        act_cals = day["calories_data"]["net_activity_calories"]
        avg_strs = day["stress_data"]["avg_stress_level"]
        avg_heart_rate.append(ahr)
        act_seconds.append(act_secs)
        act_calories.append(act_cals)
        avg_stress.append(avg_strs)
    df = pd.DataFrame({
        "avg_heart_rate": avg_heart_rate,
        "act_seconds": act_seconds,
        "act_calories": act_calories,
        "avg_stress": avg_stress,
    })
    return df

df = get_df_data(daily_resp_json)
df

KeyError: 'data'

In [49]:
avg_heart_rate_mean = df["avg_heart_rate"].mean()
df["avg_heart_rate"].fillna(value=avg_heart_rate_mean, inplace=True)
df

,avg_heart_rate,act_seconds,act_calories,avg_stress
0,77.917355,7770,122,56
1,67.004980,3280,60,28
2,71.544506,12325,394,34
3,71.190337,7705,339,33
4,75.495542,0,0,-1
5,75.495542,0,0,-1
6,72.600000,60,0,46
7,82.864012,17345,875,44
8,73.738278,7345,445,31
9,87.104869,6075,162,63


In [51]:
df["avg_heart_rate"].to_list()

[77.91735537190083,
 67.00498007968127,
 71.54450625869264,
 71.19033674963397,
 75.49554216433334,
 75.49554216433334,
 72.6,
 82.86401202103681,
 73.73827791986359,
 87.10486891385767]

In [ ]:
# dict_keys(['MET_data', 'active_durations_data', 'calories_data', 'device_data', 'distance_data', 'heart_rate_data', 'metadata', 'oxygen_data', 'scores', 'strain_data', 'stress_data', 'tag_data'])

In [47]:
daily_resp_json["data"][0]["stress_data"]

{'activity_stress_duration_seconds': 5580,
 'avg_stress_level': 56,
 'high_stress_duration_seconds': 1680,
 'low_stress_duration_seconds': 3840,
 'max_stress_level': 94,
 'medium_stress_duration_seconds': 2820,
 'rest_stress_duration_seconds': 120,
 'samples': [],
 'stress_duration_seconds': 8340}